# Setup

In [1]:
import os

import numpy as np
import pandas as pd

from pprint import pprint

from zip import unzip_archive
from fedot_util import run_example
from llm_util import run_web_model_multicall
from web_api import WebAssistant
from data import Dataset
import prompts

# Загрузка данных

In [2]:
dataset_path = [
    'titanic', 
    'credit-g'
][0]
dataset_path = os.sep.join(['datasets', dataset_path])

# zip_filename = f"{dataset_path}.zip"
# os.makedirs(dataset_path, exist_ok=True)
# unzip_archive(zip_filename, dataset_path)

In [5]:
dataset = Dataset.load_from_path(dataset_path)
dataset_description = dataset.get_description()
dataset_metadata_description = dataset.get_metadata_description()

print(dataset_metadata_description)

name: titanic passengers survival 
 description: passengers survived the Titanic shipwreck 
 train_split_name: train 
 splits: dict_keys(['train', 'test_X', 'test_y'])


# Запрос к web-модели

In [6]:
task_prompts = {
    "categorical_columns": {
        "system": dataset_description,
        "task": prompts.categorical_definition_prompt,
        "context": prompts.categorical_definition_context,
    },
    "target_column": {
        "system": dataset_description,
        "task": prompts.target_definition_prompt,
        "context": None,
    },
    "task_type": {
        "system": dataset_description,
        "task": prompts.task_definition_prompt,
        "context": None,
    }
}

#Выбор модели

model_type = ["8b", "70b"][0]
url = "http://10.32.2.2:8672/v1/chat/completions"

if model_type == "70b":
    url = "http://10.32.15.21:6672/generate"

model = WebAssistant(url)
responses = run_web_model_multicall(
    model, task_prompts
)
pprint(responses)

ImportError: cannot import name 'task_definition_prompts' from 'prompts' (c:\Users\Stas\Documents\python\AutoML-LLM\prompts.py)

# Запуск фреймворка

In [4]:

if dataset_path == 'titanic':
    test_df = dataset_metadata["splits"]["test_X"].merge(dataset_metadata["splits"]["test_y"],
                                                         on='PassengerId', how='inner')
else:
    test_df = dataset_metadata["splits"]["test"]

train_df = dataset_metadata["splits"]["train"]

prediction = run_example(train_df = train_df, test_df = test_df,
                          dataset_metadata = dataset_metadata)

Generations:   0%|          | 0/10000 [02:16<?, ?gen/s]


{'roc_auc': 0.941, 'accuracy': 0.828}


In [5]:
prediction[:5]

array([[0],
       [0],
       [0],
       [0],
       [1]], dtype=int64)

In [ ]:
result_df = pd.DataFrame(prediction, columns=[dataset_metadata["target_column"]])

result_df.to_csv(f"{dataset_path}/predictions.csv")